<h1>TF-IDF</h1>
Author: Tristan

You can learn about TF-IDF here:
https://en.wikipedia.org/wiki/Tf–idf
In brief, it is a way to choose the weight of different terms in a corpus.

It is *not* a way to reduce the number of terms.
Terms can be reduced afterwards using Semantic Analysis (for which there are multiple methods)
https://en.wikipedia.org/wiki/Semantic_analysis_(machine_learning)

In [52]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import *
import scipy

In [7]:
import nlp 
data = nlp.proc_text()
data.head()

,index,title,text,text_noim,links,process
0,3,Art,[[File:Chemin montant dans les hautes herbes -...,]]\n[[Non-verbal communication]]\n[[Basic Eng...,"[Non-verbal communication, Basic English 850 w...",non verbal commun basic english word
1,19,Abbreviation,An '''abbreviation''' is a shorter way to writ...,An '''abbreviation''' is a shorter way to writ...,"[English language, apostrophe, Period (punctua...",abbrevi shorter way write word phrase peopl us...
2,29,Astronomy,[[File:Atlas Coelestis-1.jpg|thumb|280px|18th ...,]],[],
3,50,Browser,{{Distinguish|Web browser}}\n[[Image:Giraffe f...,{{Distinguish|Web browser}}\n[[Image:Giraffe f...,"[Image:Giraffe feeding, Tanzania crop.jpg|thum...",distinguish web browser imag giraff feed tanza...
4,63,Bubonic plague,{{Infobox disease\n| Name = Bubonic...,{{Infobox disease\n| Name = Bubonic...,"[Lymphatic system|lymph, plague, bacterium, Ye...",infobox diseas name bubon plagu imag plagu bub...


Looks like the cleaning isn't yet right

In [31]:
vectorizer = TfidfVectorizer(analyzer = "word", max_df=1.0, min_df=1)
#max_df is the fraction of documents that must have a word before it is ignored.
#min_df is the number of documents that must have a word for it to be considered.
#norm="l2" normalizes each document vector to a (pythagorean) length of 1.
clean_text = data["process"]
weighted_words = vectorizer.fit_transform(clean_text)

In [32]:
#also get the bag of words without weighting for comparison
vectorizer2 = CountVectorizer(analyzer = "word")
unweighted_words = vectorizer2.fit_transform(clean_text)

In [91]:
#Want a function to list words alongside their frequency
def get_word_frequency(sparse_matrix,doc,word_list):
    #find number of distinct words in given document
    num_words = sparse_matrix[doc,:].getnnz()
    #initialize DataFrame
    word_frequency = pd.DataFrame(index=range(num_words), columns=['word','frequency'])
    #convert to another kind of sparse matrix
    cx = scipy.sparse.coo_matrix(sparse_matrix[doc,:])
    #Loop over nonzero elements in the sparse matrix
    #with i = column number, j = weight, and k being the appropriate row of the DataFrame
    for i,j,k in zip(cx.col,cx.data,range(num_words)):
        word_frequency['word'][k] = word_list[i]
        word_frequency['frequency'][k] = j
        
    #Finally, sort the DataFrame
    word_frequency.sort_values('frequency',inplace=True,ascending=False)
    return word_frequency

In [97]:
doc_number = 10
test_freq = get_word_frequency(unweighted_words,doc_number,vectorizer2.get_feature_names())
test_weight = get_word_frequency(weighted_words,doc_number,vectorizer.get_feature_names())
print(str(doc_number) + "th document, before TF-IDF:\n",test_freq[0:10])
print("After TF-IDF:\n",test_weight[0:10])

10th document, before TF-IDF:
         word frequency
22  februari        14
24     month         9
4        day         8
3       year         7
48      week         3
63    presid         3
64      born         3
26      leap         3
43    zodiac         2
54  shortest         2
After TF-IDF:
         word frequency
54  februari  0.498278
52     month  0.439532
72       day  0.300026
50      leap  0.217643
30   aquariu  0.182064
33    zodiac  0.167763
73      year  0.161364
36    violet  0.159397
28      week  0.154104
22  shortest  0.147851
